In [1]:
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-bigquery-storage
!pip install --upgrade pyarrow
!pip install --upgrade google-cloud-core
!pip install --upgrade chart_studio

     |████████████████████████████████| 174kB 2.7MB/s 
     |████████████████████████████████| 92kB 5.7MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 92kB 4.9MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.17.2, but you'll have google-auth 1.20.1 which is incompatible.
ERROR: google-cloud-storage 1.18.1 has requirement google-resumable-media<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.7.1 which is incompatible.
  Found existing installation: google-auth 1.17.2
    Uninstalling google-auth-1.17.2:
      Successfully uninstalled google-auth-1.17.2
  Found existing installation: google-api-core 1.16.0
    Uninstalling google-api-core-1.16.0:
      Successfully uninstalled google-api-core-1.16.0
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found existing installation: google-resumable-media 0.4.1
    

     |████████████████████████████████| 143kB 2.8MB/s 


     |████████████████████████████████| 17.2MB 39.8MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
Requirement already up-to-date: google-cloud-core in /usr/local/lib/python3.6/dist-packages (1.4.1)
     |████████████████████████████████| 71kB 2.0MB/s 


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
project_id = 'tc-test-project-260312' #@param{type:"string"}

# Packages used for interfacing w/ BigQuery from Python
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

# Create BigQuery storage client
bq_storage_client = bigquery_storage_v1beta1.BigQueryStorageClient()

In [11]:
%%bigquery met_data --project {project_id}

CREATE MODEL `bqml.the_met_modelv3`
OPTIONS(model_type='logistic_reg') AS

SELECT IF(ARRAY_LENGTH(faceAnnotations)=0 OR faceAnnotations IS NULL, 0, 1) as label
  , IFNULL(department, "") department
  , IFNULL(culture, "") culture
  , IFNULL(classification, "") classification
  , is_highlight
  , is_public_domain
  , object_begin_date
  , object_end_date
FROM
  `bigquery-public-data.the_met.objects` o
INNER JOIN
  `bigquery-public-data.the_met.vision_api_data` data
ON
  o.object_id = data.object_id
WHERE
  MOD(FARM_FINGERPRINT(o.object_number),5) != 0

In [13]:
%%bigquery met_data_predict --project {project_id}

SELECT
  *
FROM
  ML.EVALUATE(
    MODEL `bqml.the_met_modelv3`,
    (
      SELECT IF(ARRAY_LENGTH(faceAnnotations)=0 OR faceAnnotations IS NULL, 0, 1) as label
        , IFNULL(department, "") department
        , IFNULL(culture, "") culture
        , IFNULL(classification, "") classification
        , is_highlight
        , is_public_domain
        , object_begin_date
        , object_end_date
      FROM
        `bigquery-public-data.the_met.objects` o
      INNER JOIN
        `bigquery-public-data.the_met.vision_api_data` data
      ON
        o.object_id = data.object_id
      WHERE
        MOD(FARM_FINGERPRINT(o.object_number),5) = 0
    )
  )

In [14]:
met_data_predict

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.600592,0.243756,0.903845,0.346771,0.244465,0.864616
